In [1]:
pip install selenium

In [2]:
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.by import By
import time
import pandas as pd

In [11]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
import pandas as pd
import time

url = input("Masukkan URL toko: ")
pages = int(input("Masukkan jumlah halaman review: "))

driver = webdriver.Chrome()
driver.get(url)

produk, rating, review = [], [], []

for _ in range(pages):
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    for container in soup.select('article.css-1pr2lii'):
        try:
            review.append(container.select_one('[data-testid=lblItemUlasan]').text)
            produk.append(container.select_one('p[data-unify=Typography]').text)
            stars = container.select('svg[fill="#FFD45F"]')
            rating.append(len(stars))
        except:
            continue
    time.sleep(2)
    try:
        next_btn = driver.find_element(By.CSS_SELECTOR, "button[aria-label^='Laman berikutnya']")
        next_btn.click()
    except:
        break
    time.sleep(2)

driver.quit()
df = pd.DataFrame({'Produk': produk, 'Rating': rating, 'Ulasan': review})


In [12]:
df

,Produk,Rating,Ulasan
0,Ventela Victory Low Grey [ORIGINAL] - Abu,0,Sesuai pesanan.... Trimakasih
1,Ventela Olympic Low White Brown [ORIGINAL] - P...,0,A
2,Ventela Ethnic High Black Natural [ORIGINAL] -...,0,Barang sesuai dan sangat bagus
3,Ventela Public Low Black Natural [ORIGINAL] - ...,0,Bagus
4,Ventela Alpha Low Cream [ORIGINAL],0,"kaki ku ukuran 25cm, cekout 39, kegedean sii c..."
...,...,...,...
1473,Ventela Republic Low White [ORIGINAL] - Putih,0,Kondisi bagus
1474,Ventela Ethnic Low Black Natural [ORIGINAL] - ...,0,"Mantap sesuai ekspektasi, local pride"
1475,Ventela Basic High Black Natural [ORIGINAL] - ...,0,Bagusssssw
1476,Ventela Ethnic Low All Black [ORIGINAL] - Hitam,0,semoga awet


In [ ]:
# For Product Analysis
url = input("Masukkan url produk : ")
pages = int(input("Masukkan jumlah halaman review :"))

driver = webdriver.Chrome()
driver.get(url)

review = []
rating = []
for i in range (0, pages-1):
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    containers = soup.findAll('article', attrs = {'class':'css-15m2bcr'})

    for container in containers:
        try:
            data = container.find('span', attrs = {'data-testid':'lblItemUlasan'}).text
            review.append(data)
            data2 = container.find_all('svg')
            star = sum(1 for star in data2 if '#FFD45F' in star.get('fill'))
            rating.append(star)
        except AttributeError:
            continue

    time.sleep(2)
    driver.find_element(By.CSS_SELECTOR, "button[aria-label^='Laman berikutnya']").click()
    time.sleep(3)

df = pd.DataFrame({'Rating': rating, 'Ulasan': review})

In [11]:
df

,Rating,Ulasan
0,5,Packing standar. Pengiriman cepat. Sesuai pesa...
1,5,"bagus nyaman, tapi pilih abu abu ternyata warn..."
2,5,"pas sampe kardusnya penyok, untungnya sepatu a..."
3,5,"Ukuran pas sesuai deskripsi, jahitan & lem sep..."
4,5,"ukuran pas, bahan ok. dipakai empuk nyaman . r..."
...,...,...
475,5,"Baguss banget sepatunyaa.. Ukurannya pas, sesu..."
476,5,Keren bagus banget sepatunya local pride!
477,4,"ukuran pas, packaging rapi, tapi pengiriman lu..."
478,5,pesanan sudah sampai dengan kondisi baik\nsepa...


In [13]:
df.to_csv("ventela.csv", index=False)

In [14]:
pip install nltk gensim pyLDAvis

🎯 Tujuan Proyek:

Analisis sentimen tiap review → beri label (positif/negatif/netral)

Topic modeling → pahami topik apa yang sering muncul dalam review positif/negatif

Insight → topik apa yang sering bikin pelanggan puas/tidak puas?

In [28]:
# Import Library
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import gensim
from gensim import corpora
import pyLDAvis.gensim_models as gensimvis
import pyLDAvis
import pandas as pd

# Download stopwords jika belum
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('punkt_tab')

# =====================================
# 1. Load Data
# =====================================
# Misal: file berisi kolom 'review'
df = pd.read_csv('compass.csv')
df = df.dropna(subset=['Ulasan'])  # hilangkan review kosong
df

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


,Produk,Rating,Ulasan
0,Sepatu Compass Tribune Away Maroon,0,Mewah banget walaupun sepatu lokal ga kalah sa...
1,Sepatu Compass Gazelle Hi Cream,0,emang ga salah si product compass kualitasnya ...
2,Sepatu Compass Tribune White Blue,0,Barang aman dan sudah pasti ori dijamin nfc-ny...
3,Sepatu Compass Tribune Death Star,0,2 nya sama wkwk
4,Sepatu Compass Gazelle Low Cream,0,Kerennn
...,...,...,...
2053,Sepatu Compass Gazelle Low Cream,0,Ukuran 39 di sini panjang kakinya 26 cm. Udah ...
2054,Sepatu Compass Velocity Black,0,Baik
2055,Compass Gazelle Low Black Gum,0,"pengiriman cepat, adminnya ramah banget, minta..."
2056,Sepatu Compass Retrograde Hi Decon Triple Black,0,compas emang keren😍


In [29]:
# =====================================
# 2. Preprocessing Text
# =====================================
# Ambil stopwords Bahasa Indonesia dari NLTK
stop_words = set(stopwords.words('indonesian'))

# Tambahkan custom stopwords (singkatan dan kata tidak penting lainnya)
custom_stopwords = {'bgt', 'yg', 'aja', 'dong', 'nih', 'nya', 'deh', 'gk', 'ga', 'tp', 'udh', 'dr','banget', 'it', 'beli', 'kali', 'pol', 'trs', 'bbrp',
                    'gak', 'udah', 'sumpah', 'paa'}
stop_words.update(custom_stopwords)

def preprocess_text(text):
    text = text.lower()
    tokens = word_tokenize(text)
    tokens = [t for t in tokens if t.isalpha()]  # buang angka & simbol
    tokens = [t for t in tokens if t not in stop_words]
    return tokens

df['tokens'] = df['Ulasan'].apply(preprocess_text)


In [30]:
# =====================================
# 3. Membuat Dictionary dan Corpus
# =====================================
dictionary = corpora.Dictionary(df['tokens'])
corpus = [dictionary.doc2bow(text) for text in df['tokens']]

In [31]:
# =====================================
# 4. LDA Modeling
# =====================================
lda_model = gensim.models.LdaModel(
    corpus=corpus,
    id2word=dictionary,
    num_topics=4,
    random_state=100,
    update_every=1,
    chunksize=100,
    passes=10,
    alpha='auto',
    per_word_topics=True
)


In [32]:
# =====================================
# 5. Melihat Topik dan Kata Kunci
# =====================================
topics = lda_model.print_topics(num_words=5)
for i, topic in topics:
    print(f"Topik #{i+1}: {topic}")


Topik #1: 0.091*"keren" + 0.075*"ukuran" + 0.059*"pas" + 0.043*"nyaman" + 0.031*"warna"
Topik #2: 0.160*"bagus" + 0.044*"sepatu" + 0.034*"good" + 0.020*"sepatunya" + 0.015*"cocok"
Topik #3: 0.056*"compass" + 0.036*"size" + 0.027*"oke" + 0.026*"semoga" + 0.024*"aman"
Topik #4: 0.089*"sesuai" + 0.067*"mantap" + 0.052*"barang" + 0.045*"pengiriman" + 0.042*"cepat"


In [33]:
# =====================================
# 6. Visualisasi dengan pyLDAvis
# =====================================
pyLDAvis.enable_notebook()
vis = gensimvis.prepare(lda_model, corpus, dictionary)
pyLDAvis.save_html(vis, 'compass.html')
print("Visualisasi berhasil disimpan di 'lda_kanky_positive.html'")

Visualisasi berhasil disimpan di 'lda_kanky_positive.html'
